In [1]:
import math
from openai import OpenAI

1. Detect high-level topic (zero-shot)
2. Detect targets (zero-shot)
3. If confidence is lower than threshold, generate new target (gpt assistant)
4. Generate stance to the targets (gpt assistant) -> need confidence scores

In [1]:
import json
import pandas as pd
import time
from tqdm import tqdm
from collections import defaultdict
import re
import ast
import numpy as np
import random
import pickle

### Comparing topics


In [56]:
# migration VS security
client = OpenAI(api_key="sk-proj-3KhQdlgMCuoQc6EL2y1ZT3BlbkFJYHG9JgPGpOLsRnDSKuSs")
topic_comparison_id = "asst_q4WpyeJSDfBIDnBojvr55Mf7"
topic_comparison_loose_id = "asst_Z40gSwa21dlftmKSaXptTIcj"

In [40]:
def run_topic_comparison(text, statements):
    statement = "Text: " + str(text) + "\nGroup of statements: " + str(statements) + "\nAnswer: "
    
    if str(statement) not in ["NaN", "nan"]:
        try:
            # Create a thread with a message.
            thread = client.beta.threads.create(
                messages=[
                    {
                        "role": "user",
                        # Update this with the query you want to use.
                        "content": statement,
                    }
                ]
            )

            # Submit the thread to the assistant (as a new run).
            run = client.beta.threads.runs.create(thread_id=thread.id, assistant_id=topic_comparison_id)

            # Wait for run to complete.
            while run.status != "completed":
                run = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)
                time.sleep(1)
            # else:
            #     print(f"🏁 Run Completed!")

            # Get the latest message from the thread.
            message_response = client.beta.threads.messages.list(thread_id=thread.id)
            messages = message_response.data

            # Print the latest message.
            latest_message = messages[0]
            
            return latest_message.content[0].text.value
        
        except:
            print(f"Run went wrong")
            return None

In [57]:
def run_topic_comparison_loose(text, statements):
    statement = "Text: " + str(text) + "\nGroup of statements: " + str(statements) + "\nAnswer: "
    
    if str(statement) not in ["NaN", "nan"]:
        try:
            # Create a thread with a message.
            thread = client.beta.threads.create(
                messages=[
                    {
                        "role": "user",
                        # Update this with the query you want to use.
                        "content": statement,
                    }
                ]
            )

            # Submit the thread to the assistant (as a new run).
            run = client.beta.threads.runs.create(thread_id=thread.id, assistant_id=topic_comparison_loose_id)

            # Wait for run to complete.
            while run.status != "completed":
                run = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)
                time.sleep(1)
            # else:
            #     print(f"🏁 Run Completed!")

            # Get the latest message from the thread.
            message_response = client.beta.threads.messages.list(thread_id=thread.id)
            messages = message_response.data

            # Print the latest message.
            latest_message = messages[0]
            
            return latest_message.content[0].text.value
        
        except:
            print(f"Run went wrong")
            return None

In [32]:
df_migration = pd.read_excel("data/Zurich_NLP/rephrased/migration.xlsx")
df_security = pd.read_excel("data/Zurich_NLP/rephrased/security.xlsx")
df_welfare = pd.read_excel("data/Zurich_NLP/rephrased/welfare.xlsx")
df_education = pd.read_excel("data/Zurich_NLP/rephrased/education.xlsx")

In [49]:
migration = list(df_migration['shortened_question'])
security = list(df_security['shortened_question'])
welfare = list(df_welfare['shortened_question'])
education = list(df_education['shortened_question'])

In [77]:
education_vs_welfare_loose

defaultdict(list,
            {'Harmonisierung von Lehrplänen zwischen den Kantonen (z.B. durch Lehrplan 21 oder PER) befürworten.': ['Einführung eines mehrwöchigen bezahlten Vaterschaftsurlaubs zusätzlich zur Mutterschaftsversicherung.'],
             'Dispense aus religiösen Gründen für einzelne Fächer oder Veranstaltungen (z.B. Sport- oder Sexualkundeunterricht) bewilligen.': ['Einführung eines 24-wöchigen Elternurlaubs ("Elternzeit") zusätzlich zur Mutterschaftsversicherung.'],
             'Kinder mit Lernschwierigkeiten oder Behinderungen in regulären Schulklassen unterrichten (integrative Schule) befürworten.': ["'Staatliche finanzielle Unterstützung der Fremdbetreuung von Kindern (Steuerabzüge oder Subventionen).', 'Finanzielle Unterstützung der ausserfamiliären Kinderbetreuung durch den Bund.', 'Stärkere finanzielle Unterstützung des Staates für familienergänzende Betreuungsstrukturen.', 'Einführung von gesamtschweizerischen Ergänzungsleistungen für Familien mit niedrigem Eink

In [60]:
security_vs_education = defaultdict(list)
security_vs_education_loose = defaultdict(list)
for txt in tqdm(security):
    print("Text:", txt)
    security_vs_education[txt] = []
    security_vs_education_loose[txt] = []
    #generate topics

    res = run_topic_comparison(txt, education)
    security_vs_education[txt].append(res)
    print(res)
    
    res2 = run_topic_comparison_loose(txt, education)
    security_vs_education_loose[txt].append(res2)
    print(res2)
    
with open('data/Zurich_NLP/related_topics/security_vs_education.json', 'w') as file:
    json.dump(security_vs_education, file)
with open('data/Zurich_NLP/related_topics/security_vs_education_loose.json', 'w') as file:
    json.dump(security_vs_education_loose, file)

  0%|          | 0/15 [00:00<?, ?it/s]

Text: Die Schweiz soll neue Kampfflugzeuge beschaffen.
None


  7%|▋         | 1/15 [00:08<01:54,  8.18s/it]

Keine der gegebenen Aussagen steht direkt in Verbindung mit der Beschaffung neuer Kampfflugzeuge, daher gibt es hier keine anzuzeigenden relevanten Aussagen.
Text: Die Schweizer Armee soll auf höchstens 100000 Soldaten reduziert werden.
None


 13%|█▎        | 2/15 [00:14<01:35,  7.35s/it]

...

[]
Text: Die rechtlichen Möglichkeiten zur Fahndung mittels DNA-Analyse sollen ausgeweitet werden.
None


 93%|█████████▎| 14/15 [01:44<00:07,  7.33s/it]

None of the provided statements directly relate to the expansion of legal possibilities for searching via DNA analysis. Therefore, no statements from the list match the requested criteria.
Text: nan
None


100%|██████████| 15/15 [01:57<00:00,  7.83s/it]

Harmonisierung von Lehrplänen zwischen den Kantonen (z.B. durch Lehrplan 21 oder PER) befürworten.
Dispense aus religiösen Gründen für einzelne Fächer oder Veranstaltungen (z.B. Sport- oder Sexualkundeunterricht) bewilligen.
Kinder mit Lernschwierigkeiten oder Behinderungen in regulären Schulklassen unterrichten (integrative Schule) befürworten.
Finanzielle Unterstützung des Bundes für berufliche Weiterbildung und Umschulung ausbauen.
Zweite Landessprache in der Primarschule in allen Kantonen unterrichten.
Staat stärker für gleiche Bildungschancen einsetzen (z.B. mit Nachhilfe-Gutscheinen für Schüler/-innen aus Familien mit geringem Einkommen).
Wirtschaftlichen Nutzen von Forschungsprojekten bei der Vergabe von Fördergeldern des Bundes stärker berücksichtigen.


In [59]:
security_vs_welfare = defaultdict(list)
security_vs_welfare_loose = defaultdict(list)
for txt in tqdm(security):
    print("Text:", txt)
    security_vs_welfare[txt] = []
    security_vs_welfare_loose[txt] = []
    #generate topics

    res = run_topic_comparison(txt, welfare)
    security_vs_welfare[txt].append(res)
    print(res)
    
    res2 = run_topic_comparison_loose(txt, welfare)
    security_vs_welfare_loose[txt].append(res2)
    print(res2)
    
with open('data/Zurich_NLP/related_topics/security_vs_welfare.json', 'w') as file:
    json.dump(security_vs_welfare, file)
with open('data/Zurich_NLP/related_topics/security_vs_welfare_loose.json', 'w') as file:
    json.dump(security_vs_welfare_loose, file)

  0%|          | 0/15 [00:00<?, ?it/s]

Text: Die Schweiz soll neue Kampfflugzeuge beschaffen.
None


  7%|▋         | 1/15 [00:12<02:59, 12.80s/it]

None of the statements provided are directly relevant or likely to influence the opinion of someone specifically concerning the topic of Switzerland acquiring new fighter jets. The provided statements predominantly focus on social welfare, family support policies, and retirement issues, which do not directly correlate with military procurement or defense policy decisions such as the acquisition of new combat aircraft. Thus, there are no statements in the list that can be definitively linked to someone's stance on the main text about Switzerland acquiring new fighter jets.
Text: Die Schweizer Armee soll auf höchstens 100000 Soldaten reduziert werden.
None


 13%|█▎        | 2/15 [00:21<02:13, 10.24s/it]

...

[]
Text: Die rechtlichen Möglichkeiten zur Fahndung mittels DNA-Analyse sollen ausgeweitet werden.
None


 93%|█████████▎| 14/15 [02:31<00:08,  8.20s/it]

None of the provided statements are directly related or relevant to the text concerning the expansion of legal measures for DNA-based searches (DNA-Analyse). Therefore, none of the statements listed would be likely to elicit an opinion based specifically on having a stance on the DNA analysis text.
Text: nan
None


100%|██████████| 15/15 [02:39<00:00, 10.64s/it]

Keine IV-Renten bei nicht objektiv nachweisbaren Schmerzstörungen (z.B. Schleudertrauma).


In [58]:
education_vs_welfare = defaultdict(list)
education_vs_welfare_loose = defaultdict(list)
for txt in tqdm(education):
    print("Text:", txt)
    education_vs_welfare[txt] = []
    education_vs_welfare_loose[txt] = []
    #generate topics

    res = run_topic_comparison(txt, welfare)
    education_vs_welfare[txt].append(res)
    print(res)
    
    res2 = run_topic_comparison_loose(txt, welfare)
    education_vs_welfare_loose[txt].append(res2)
    print(res2)
    
with open('data/Zurich_NLP/related_topics/education_vs_welfare.json', 'w') as file:
    json.dump(education_vs_welfare, file)
with open('data/Zurich_NLP/related_topics/education_vs_welfare_loose.json', 'w') as file:
    json.dump(education_vs_welfare_loose, file)

  0%|          | 0/10 [00:00<?, ?it/s]

Text: Harmonisierung von Lehrplänen zwischen den Kantonen (z.B. durch Lehrplan 21 oder PER) befürworten.
None


 10%|█         | 1/10 [00:07<01:10,  7.85s/it]

Einführung eines mehrwöchigen bezahlten Vaterschaftsurlaubs zusätzlich zur Mutterschaftsversicherung.
Text: Dispense aus religiösen Gründen für einzelne Fächer oder Veranstaltungen (z.B. Sport- oder Sexualkundeunterricht) bewilligen.
None


 20%|██        | 2/10 [00:15<01:02,  7.80s/it]

...

Keine IV-Renten bei nicht objektiv nachweisbaren Schmerzstörungen (z.B. Schleudertrauma).
Text: nan
None


 90%|█████████ | 9/10 [01:29<00:09,  9.25s/it]

Keine IV-Renten bei nicht objektiv nachweisbaren Schmerzstörungen (z.B. Schleudertrauma).
Text: nan
None


100%|██████████| 10/10 [01:36<00:00,  9.61s/it]

nan


In [55]:
migration_vs_education = defaultdict(list)
for txt in tqdm(migration):
    print("Text:", txt)
    migration_vs_education[txt] = []
    #generate topics

    res = run_topic_comparison(txt, education)
    migration_vs_education[txt].append(res)
    print(res)
    
with open('data/Zurich_NLP/related_topics/migration_vs_education.json', 'w') as file:
    json.dump(migration_vs_education, file)

  0%|          | 0/13 [00:00<?, ?it/s]

Text: Unterstützung des bestehenden Personenfreizügigkeitsabkommens mit der EU.


  8%|▊         | 1/13 [00:03<00:39,  3.28s/it]

None
Text: Weitere Verschärfung des Asylrechts.


 15%|█▌        | 2/13 [00:06<00:37,  3.40s/it]

...

Staat stärker für gleiche Bildungschancen einsetzen (z.B. mit Nachhilfe-Gutscheinen für Schüler/-innen aus Familien mit geringem Einkommen).
Text: Stimm- und Wahlrecht auf Gemeindeebene für Ausländer/-innen die seit mindestens zehn Jahren in der Schweiz leben.


 92%|█████████▏| 12/13 [00:40<00:03,  3.36s/it]

None
Text: Vereinfachte Erteilung eines geregelten Aufenthaltsstatus für Sans-Papiers.


100%|██████████| 13/13 [00:43<00:00,  3.34s/it]

None.


In [54]:
with open('data/Zurich_NLP/related_topics/migration_vs_welfare_loose.json', 'w') as file:
    json.dump(migration_vs_welfare, file)

In [48]:
migration_vs_security = defaultdict(list)
for txt in tqdm(migration):
    print("Text:", txt)
    migration_vs_security[txt] = []
    #generate topics

    res = run_topic_comparison(txt, security)
    migration_vs_security[txt].append(res)
    print(res)

  0%|          | 0/13 [00:00<?, ?it/s]

Text: Unterstützung des bestehenden Personenfreizügigkeitsabkommens mit der EU.


  8%|▊         | 1/13 [00:04<00:54,  4.54s/it]

Die Schweiz soll das Schengen-Abkommen kündigen und verstärkte Personenkontrollen an der Grenze einführen.
Text: Weitere Verschärfung des Asylrechts.


 15%|█▌        | 2/13 [00:09<00:49,  4.49s/it]

...

None
Text: Stimm- und Wahlrecht auf Gemeindeebene für Ausländer/-innen die seit mindestens zehn Jahren in der Schweiz leben.


 92%|█████████▏| 12/13 [01:24<00:09,  9.33s/it]

None
Text: Vereinfachte Erteilung eines geregelten Aufenthaltsstatus für Sans-Papiers.


100%|██████████| 13/13 [01:27<00:00,  6.74s/it]

None


In [51]:
with open('data/Zurich_NLP/related_topics/migration_vs_security.json', 'w') as file:
    json.dump(migration_vs_security, file)

In [38]:
security

['Die Schweiz soll neue Kampfflugzeuge beschaffen.',
 'Die Schweizer Armee soll auf höchstens 100000 Soldaten reduziert werden.',
 'Der Armeebestand soll von 190000 auf 80000 Soldaten reduziert werden.',
 'Das "Raser"-Gesetz soll gelockert werden.',
 'Die Armee soll polizeiliche Aufgaben im Innern wahrnehmen.',
 'Die verschärften Regeln zum Erwerb und Besitz von Waffen sollen beibehalten werden.',
 'Die Verschärfung der Zulassungsbedingungen zum Zivildienst soll nicht umgesetzt werden.',
 'Das Jugendstrafrecht soll längere Haftstrafen in geschlossenen Anstalten statt Resozialisierungsmassnahmen bevorzugen.',
 'Der Konsum und Besitz von weichen und harten Drogen für den Eigengebrauch soll legalisiert werden.',
 'Die Schweiz soll das Schengen-Abkommen kündigen und verstärkte Personenkontrollen an der Grenze einführen.',
 'Der Export von Kriegsmaterial aus der Schweiz soll verboten werden.',
 'Schweizer Jugendliche sollen frei zwischen Militärdienst und zivilem Ersatzdienst wählen können.

### Comparing statements from related topics

In [116]:
# migration VS security
client = OpenAI(api_key="sk-proj-3KhQdlgMCuoQc6EL2y1ZT3BlbkFJYHG9JgPGpOLsRnDSKuSs")
inconsistency_detection_id = "asst_5B5qrEzQpyKvI6E3piJV8Oji"

In [117]:
def run_inconsistency_detection(text1, text2):
    statement = "Text 1: " + str(text1) + "\nText 2: " + str(text2) + "\nAnswer: "
    
    if str(statement) not in ["NaN", "nan"]:
        try:
            # Create a thread with a message.
            thread = client.beta.threads.create(
                messages=[
                    {
                        "role": "user",
                        # Update this with the query you want to use.
                        "content": statement,
                    }
                ]
            )

            # Submit the thread to the assistant (as a new run).
            run = client.beta.threads.runs.create(thread_id=thread.id, assistant_id=inconsistency_detection_id)

            # Wait for run to complete.
            while run.status != "completed":
                run = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)
                time.sleep(1)
            # else:
            #     print(f"🏁 Run Completed!")
                
                            # Get the latest message from the thread.
            message_response = client.beta.threads.messages.list(thread_id=thread.id)
            messages = message_response.data
            latest_message = messages[0]
            
            return latest_message.content[0].text.value
        
        except:
            print(f"Run went wrong")
            return None

In [97]:
q1 = "Soll der Bund Ausländer/-innen bei der Integration stärker unterstützen?"
q2 = "Soll sich der Staat stärker für gleiche Bildungschancen einsetzen (z.B. mit Nachhilfe-Gutscheinen für Schüler/-innen aus Familien mit geringem Einkommen)?"

In [81]:
# find all texts that support q1 but are against q2
# find all texts that support q2 but are against q1

In [82]:
with open(r"C:\Users\Nursulu_1\Downloads\xstance-v1.0.0\ZurichNLP-xstance-90283b3\data\xstance-data-v1.0\test.jsonl", 'r', encoding='utf-8') as file:
    texts = [json.loads(line) for line in file]

In [87]:
questions = [el['question'] for el in texts if el['language'] == 'de']

In [95]:
questions = set(questions)

In [100]:
texts[:3]

[{'id': 17820,
  'language': 'de',
  'question_id': 3415,
  'question': 'Sollen die Renten der Pensionskasse durch eine Senkung des Umwandlungssatzes gekürzt und an die gestiegene Lebenserwartung angepasst werden?',
  'comment': 'Es muss nach anderen Lösungen gesucht werden. Die Lebenshaltungskosten (u.a. Mieten) steigen auch weiter.',
  'label': 'AGAINST',
  'numerical_label': 25,
  'author': 'aea1e176f453',
  'topic': 'Welfare',
  'test_set': 'new_comments_defr'},
 {'id': 17821,
  'language': 'de',
  'question_id': 3416,
  'question': 'Befürworten Sie Bestrebungen in den Kantonen zur Senkung der Sozialhilfeleistungen?',
  'comment': 'Die Kantone sollen sich um Missbräuche durch Leistungsbezüger kümmern.',
  'label': 'AGAINST',
  'numerical_label': 25,
  'author': 'aea1e176f453',
  'topic': 'Welfare',
  'test_set': 'new_comments_defr'},
 {'id': 17823,
  'language': 'de',
  'question_id': 3423,
  'question': 'Soll sich der Staat stärker für gleiche Bildungschancen einsetzen (z.B. mit N

In [108]:
q1_pro = [el['comment'] for el in texts if el['question'] == q1 and el['language'] == 'de' and el['label'] == 'FAVOR']
q2_pro = [el['comment'] for el in texts if el['question'] == q2 and el['language'] == 'de' and el['label'] == 'FAVOR']
q1_contra = [el['comment'] for el in texts if el['question'] == q1 and el['language'] == 'de' and el['label'] == 'AGAINST']
q2_contra = [el['comment'] for el in texts if el['question'] == q2 and el['language'] == 'de' and el['label'] == 'AGAINST']

In [109]:
print(len(q1_pro))
print(len(q2_pro))
print(len(q1_contra))
print(len(q2_contra))

26
25
22
17


You are given two groups of statements, A and B. Are there any statements from group A that are inconsistent with any statements in group B?

In [114]:
q2_pro

['Bildung ist eines unserer Ressourcen und sollte wo immer möglich gestärkt werden.',
 'Bildung ist ein wichtiger Teil unserer Entwicklung. Daher sollen die Bildungschancen möglichst für alle gleich sein.',
 'Bildung ist das wertvollste Gut eines Staates und sollte dementsprechend gefördert werden.',
 'Gleiche Bildungschancen sind sehr wichtig! Förderung, besonders für Kinder mit Schul-Schwächen aus ärmeren Familien, bedeutet auch, dass diese Kinder so bessere Chancen haben einen Beruf zu erlernen, der sie aus dem Kreislauf der "Armut" herausholt.',
 'Ist-Zustand der Chancenverteilung ist ungenügend. Führt zu vielen privilegierten Erben mit wenig Ahnung, viel Selbstüberschätzung an der Macht und immer öfter zu absoluten Nichtskönnern in Führungspositionen.',
 'Die soziale Mobilität ist in der Schweiz tiefer als allgemein angenommen, das muss sich ändern!',
 'Nachhilfegutscheine ja, aber nur für interessierte und leistungswillige Kinder.',
 'Für mich essenziell um fehlende Förderung zu 

In [115]:
q1_contra

['Integration muss willentlich geschehen, einerseits von seiten der Bevölkerung, andererseits von den Betroffenen. Dann braucht es keine staatliche Unterstützung.',
 'Wer in die Schweiz kommt und mit unserer Kultur lebt, ist herzlich willkommen! Intergration ist eine Bringschuld seitens der Einwanderer, zumal die Schweiz sehr viele Chancen zur aktiven Intergration bietet!',
 'Für die Integration von Ausländer wird bereits viel geboten, Integration ist primär Aufgabe des zu Integrierenden.',
 'Integration ist gut und wichtig. Jedoch braucht es nun mehr auf Kantonaler- und Gemeindebene. Evt auch Unterstützung für Kirchgemeinden, die einen Teil übernehmen könnten.',
 'Ausnahme: Zusätzliche Förderung von Sprachkenntnissen und des Verständnisses unserer Demokratie und Gesellschaft.',
 'Zur Zeit bestehen genügend Massnahmen. Es braucht keinen weiteren Ausbau. Fördern und Fordern.',
 'Jeder Ausländer ist verpflichtet, sich zu integrieren, Integration ist eine Holplicht.',
 'Ich sehe dies eher

In [ ]:
q1

### Getting data

In [3]:
# texts = pd.read_csv("data/perspectrum_test_filtered.csv")

In [7]:
with open(r"C:\Users\Nursulu_1\Downloads\xstance-v1.0.0\ZurichNLP-xstance-90283b3\data\xstance-data-v1.0\test.jsonl", 'r', encoding='utf-8') as file:
    texts = [json.loads(line) for line in file]

In [8]:
# with open("data/Zurich_NLP/labels_Education_de", 'r') as file:
#     labels = json.load(file)

In [22]:
texts = pd.read_excel("data/Rephrased_Zurich_NLP.xlsx")

In [24]:
# orig_quest = re.sub(r"[{},']", "", texts['Original question'][0])
# orig_quest = orig_quest.split("\n")
# orig_quest = [el.strip() for el in orig_quest]

# rephrased = re.sub(r"[{},']", "", texts['Rephrased by ChatGPT'][0])
# rephrased = rephrased.split("\n")
# rephrased = [el.strip() for el in rephrased]

# shortened = re.sub(r"[{},']", "", texts['Shortened'][0])
# shortened = shortened.split("\n")
# shortened = [el.strip() for el in shortened]

# data = {'original_question': orig_quest, 'rephrased_question': rephrased, 'shortened_question': shortened}
# df = pd.DataFrame.from_dict(data, orient='index').transpose()
# df = pd.DataFrame.from_dict(data, orient='index').transpose()

# df.to_excel("data/Zurich_NLP/rephrased/welfare.xlsx")

In [6]:
with open(r"C:\Users\Nursulu_1\Downloads\xstance-v1.0.0\ZurichNLP-xstance-90283b3\data\xstance-data-v1.0\test.jsonl", 'r', encoding='utf-8') as file:
    texts = [json.loads(line) for line in file]
texts = [el['comment'] for el in texts if el['topic'] == "Education" and el['language'] == 'de']

In [7]:
data = pd.DataFrame({"text": texts})

In [8]:
data['Forced_stances'] = 0
data['Vectors'] = 0

### List of assistants

In [9]:
client = OpenAI(api_key="sk-proj-3KhQdlgMCuoQc6EL2y1ZT3BlbkFJYHG9JgPGpOLsRnDSKuSs")
stance_detection_id = "asst_snO9IXRwRd5ugUihoKNzflOU"

In [11]:
data = data[:100]

In [12]:
def run_stance_detection(targets, text):
    statement = "Targets: " + str(targets) + "\nText: " + text + "\nStances: "
    
    if str(statement) not in ["NaN", "nan"]:
        try:
            # Create a thread with a message.
            thread = client.beta.threads.create(
                messages=[
                    {
                        "role": "user",
                        # Update this with the query you want to use.
                        "content": statement,
                    }
                ]
            )

            # Submit the thread to the assistant (as a new run).
            run = client.beta.threads.runs.create(thread_id=thread.id, assistant_id=stance_detection_id)

            # Wait for run to complete.
            while run.status != "completed":
                run = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)
                time.sleep(1)
            # else:
            #     print(f"🏁 Run Completed!")

            # Get the latest message from the thread.
            message_response = client.beta.threads.messages.list(thread_id=thread.id)
            messages = message_response.data

            # Print the latest message.
            latest_message = messages[0]
            
            return latest_message.content[0].text.value
        
        except:
            print(f"Run went wrong")
            return None

#### Calling ChatGPT API

In [13]:
def postprocess_answer_gpt(output):
    
    try:
        output = eval(output)
        if isinstance(output, list):
            return output
        output = list(output.keys())
        return output
    except:
        try:
            output = output.replace("\n", "")
            output = output.replace("Targets: ", "")
            output = eval(output)
            output = list(output.keys())
            return output
        except:
            try:
                output = output.replace("\n", "")
                output = output.replace("Answer: ", "")
                output = eval(output)
                output = list(output.keys())
                return output
                
            except:
                print("Error while extracting topics")
                print(output)
                return None

In [14]:
def extract_vectors(output):
    try:
        # Extract the part of the string that looks like a dictionary
        dict_str = re.search(r"{.*}", output, re.DOTALL).group()
        
        # Convert the string to a dictionary safely
        stance_dict = ast.literal_eval(dict_str)
        
        array = list(stance_dict.values())
        
        return array
    except:
        try:
            # Extract the part of the string that looks like a dictionary
            dict_str = re.search(r"{.*}", output, re.DOTALL).group()
            
            dict_str =  dict_str.replace("\n", "")
            
            # Convert the string to a dictionary safely
            stance_dict = ast.literal_eval(dict_str)
            
            array = list(stance_dict.values())
            
            return array
            
            
        except:
            print("Error while extracting stances")
            print(output)
            return None

In [16]:
for i in tqdm(range(len(data['text']))):
    # print(data['Texts'][i])
    
    forced_stances = run_stance_detection(labels, data['text'][i])  
    
    data['Forced_stances'][i] = forced_stances
    
    # print("data['Forced_targets_1'][i]")
    # print(data['Forced_targets_1'][i])

    data.to_csv("results/Zurich_NLP/education.csv", index=False)
    
    print("Extracting vectors")
    
    vec = extract_vectors(str(forced_stances))
    
    data['Vectors'][i] = str(vec)
    
    data.to_csv("results/Zurich_NLP/education.csv", index=False)
    

  0%|          | 0/100 [00:00<?, ?it/s]

C:\Users\Nursulu_1\AppData\Local\Temp\ipykernel_479880\3390047629.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Forced_stances'][i] = forced_stances
C:\Users\Nursulu_1\AppData\Local\Temp\ipykernel_479880\3390047629.py:6: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '{'Harmonisierung von Lehrplänen zwischen den Kantonen (z.B. durch Lehrplan 21 oder PER) befürworten.': 1, 'Dispense aus religiösen Gründen für einzelne Fächer oder Veranstaltungen (z.B. Sport- oder Sexualkundeunterricht) bewilligen.': 0, 'Kinder mit Lernschwierigkeiten oder Behinderungen in regulären Schulklassen unterrichten (integrative Schule) befürworten.': 1, 'Finanzielle Unterstützung des Bundes für berufliche Weiterbildung und Umschulung ausbauen.': 1

Extracting vectors


C:\Users\Nursulu_1\AppData\Local\Temp\ipykernel_479880\3390047629.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Forced_stances'][i] = forced_stances
  2%|▏         | 2/100 [00:25<20:16, 12.42s/it]

Extracting vectors


...

Extracting vectors


 99%|█████████▉| 99/100 [19:25<00:10, 10.71s/it]

Extracting vectors


100%|██████████| 100/100 [19:37<00:00, 11.77s/it]

Extracting vectors


In [5]:
data = pd.read_csv("results/perspectrum_test_filtered_40.csv")

In [6]:
data

,stance,1,2,target,text,Forced_stances,Vectors
0,1,0,0,All patents should be removed on anti-retrovir...,Patents on ARVs prevent drug companies from co...,"{'Obese people, in countries with a national h...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,0,0,0,"Mind Sports Such as Chess, Quizzing and Debati...",Chess lacks the necessary physical activity,"{'Obese people, in countries with a national h...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,0,0,0,"Mind Sports Such as Chess, Quizzing and Debati...",Olympic sports are suppose to be physical,"{'Obese people, in countries with a national h...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,1,0,0,"Mind Sports Such as Chess, Quizzing and Debati...",Mind sports are some of the world's oldest and...,"{'Obese people, in countries with a national h...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,1,0,0,"Mind Sports Such as Chess, Quizzing and Debati...",Mind sports have been around for centuries and...,"{'Obese people, in countries with a national h...",NaN
...,...,...,...,...,...,...,...
481,0,0,0,The D.A.R.E. program is good for America's kid...,There is no evidence that D.A.R.E. graduates a...,"{'Obese people, in countries with a national h...","[0, 0, 0, 0, 0, 0, 0, -1, 0, 0, 0, 0, 0, 0, 0,..."
482,0,0,0,The D.A.R.E. program is good for America's kid...,"In the long run, D.A.R.E. graduates are no bet...","{'Obese people, in countries with a national h...","[0, 0, 0, 0, 0, 0, 0, -1, 0, 0, 0, 0, 0, 0, 0,..."
483,1,0,0,The D.A.R.E. program is good for America's kid...,D.A.R.E. has great goals that should be pursue...,"{'Obese people, in countries with a national h...","[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ..."
484,1,0,0,The D.A.R.E. program is good for America's kid...,D.A.R.E. has attainable goals that outweigh th...,"{'Obese people, in countries with a national h...","[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ..."


In [7]:
vectors = []
for el in data['Vectors']:
    try: 
        el = np.array(eval(el))
        if len(el) != 40:
            el = np.zeros(40)
    except:
        el = np.zeros(40)
    vectors.append(el)

In [8]:
matrix = np.array(vectors)
matrix.shape

(486, 40)

In [9]:
data.columns

Index(['stance', '1', '2', 'target', 'text', 'Forced_stances', 'Vectors'], dtype='object')

In [10]:
type(matrix[0])

numpy.ndarray

In [27]:
np.where(matrix[:, 0] == "1")

C:\Users\Nursulu_1\AppData\Local\Temp\ipykernel_62260\410178745.py:1: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  np.where(matrix[:, 0] == "1")


(array([], dtype=int64),)

In [11]:
contradictions_by_topic = {}
for i in tqdm(range(40)):
    print('topic', i, ":", labels[i])
    # Extract the target column
    target_column = matrix[:, i]
    # Find the indices of rows where the value is some value in the target column
    support_ind = np.where(target_column == 1)[0]
    against_ind = np.where(target_column == -1)[0]
    # print(support_ind)
    # print(against_ind)
    
    support = [data['text'][el] for el in support_ind]
    against = [data['text'][el] for el in against_ind]
    topic_support = data[data['text'].isin(support)]['target'].tolist()
    topic_against = data[data['text'].isin(against)]['target'].tolist()
    
    
    # Padding in case of different length
    max_length = max(len(support), len(against))
    support.extend([np.nan] * (max_length - len(support)))
    against.extend([np.nan] * (max_length - len(against)))
    topic_support.extend([np.nan] * (max_length - len(topic_support)))
    topic_against.extend([np.nan] * (max_length - len(topic_against)))
    
    # print(len(support))
    # print(len(topic_support))
    # print(len(against))
    # print(len(topic_against))

    
    new_df = pd.DataFrame({'support': support, "topic_support" : topic_support, 
                           'against': against, "topic_against": topic_against})

    contradictions_by_topic[labels[i]] = new_df
    
    # File path where the list of DataFrames will be saved
    file_path = 'results/contradictions_by_topic_perspectrum_OESD_40.pkl'

    # Save the list of DataFrames to a file
    with open(file_path, 'wb') as file:
        pickle.dump(contradictions_by_topic, file)

100%|██████████| 40/40 [00:00<00:00, 1279.95it/s]

topic 0 : Obese people, in countries with a national health service, should face higher taxes
topic 1 : High taxes are necessary for an effective state
topic 2 : Allow pharmaceutical companies to advertise directly to consumers
topic 3 : There should be a presumption in favour of publication for information held by public bodies
topic 4 : Gender quotas in the EU are advantageous for the economies of member states
topic 5 : Provide breakfast for all school students
topic 6 : Parents should be able to choose the sex of their children
topic 7 : The D.A.R.E. program is good for America's kids (K-12) 
topic 8 : I believe in manned space flight
topic 9 : Man made climate change is a myth
topic 10 : Only buildings that are historically important should be preserved
topic 11 : Should the Cell Phones Be Allowed in Schools
topic 12 : We must limit anonymous contributions from corporations and individuals to super PACs
topic 13 : Make-up is bad for us
topic 14 : Make sex education mandatory in sc

In [12]:
for topic in contradictions_by_topic:
    contradictions_by_topic[topic].to_csv(f"results/OESD_perspectrum_40/{topic}.csv")

In [13]:
contradictions_by_topic["Co2 Does Not Cause Global Warming"]

,support,topic_support,against,topic_against
0,A ban may make the problem worse,Airbrushing of women’s bodies in the media be ...,"Restrictions will not help, instead will will ...",Airbrushing of women’s bodies in the media be ...
1,Prohibiting it will make the situation even mo...,Airbrushing of women’s bodies in the media be ...,Many scientists think it does,Co2 Does Not Cause Global Warming
2,There is no scientifically valid mechanism for...,Co2 Does Not Cause Global Warming,There are other things that cause the global w...,Co2 Does Not Cause Global Warming
3,Several scientists this it is the cause.,Co2 Does Not Cause Global Warming,Because people are losing lives and our climat...,Co2 Does Not Cause Global Warming
4,7 Global warming is based on computer modellin...,Co2 Does Not Cause Global Warming,People are losing their lives because of clima...,Co2 Does Not Cause Global Warming
5,There is no evidence that 7 Global warming is ...,Co2 Does Not Cause Global Warming,The climate is changing and people are dying.,Co2 Does Not Cause Global Warming
6,Computer modelling showing 7 Global warming ma...,Co2 Does Not Cause Global Warming,There are too many flaws,Examinations are a fair way of testing our kno...
7,The 'Global warming' phenomenon could be cause...,Co2 Does Not Cause Global Warming,NaN,NaN
8,This situation we are in with global warming m...,Co2 Does Not Cause Global Warming,NaN,NaN
9,Co2 does not hold in heat!,Co2 Does Not Cause Global Warming,NaN,NaN


### Double-checking by ChatGPT

In [5]:
filtered_topics = ['Abolish standardized tests for University Admission',
                   "Cell phone radiation is safe",
                   "Examinations are a fair way of testing our knowledge",
                   "High taxes are necessary for an effective state",
                   "Julian Assange is a journalist",
                   "Make sex education mandatory in schools"
                   ]

In [17]:
dfs = []
for topic in new_filtered:
    df = pd.read_csv(f"results/OESD_perspectrum_40/{topic}.csv")
    dfs.append(df)

In [7]:
inconsistency_detector_id = "asst_9TNkN3fPV4dybff9wzGhB1gA"

In [14]:
def run_inconsistency_detection(text1, text2):
    if str(text1) not in ["NaN", "nan"] and str(text2) not in ["NaN", "nan"]:
        statement = "Text 1: " + text1 + "\nText 2: " + text2
        try:
            # Create a thread with a message.
            thread = client.beta.threads.create(
                messages=[
                    {
                        "role": "user",
                        # Update this with the query you want to use.
                        "content": statement,
                    }
                ]
            )
        # Submit the thread to the assistant (as a new run).
            run = client.beta.threads.runs.create(thread_id=thread.id, assistant_id=inconsistency_detector_id)

            # Wait for run to complete.
            while run.status != "completed":
                run = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)
                time.sleep(1)
            # else:
            #     print(f"🏁 Run Completed!")

            # Get the latest message from the thread.
            message_response = client.beta.threads.messages.list(thread_id=thread.id)
            messages = message_response.data

            # Print the latest message.
            latest_message = messages[0]
            return latest_message.content[0].text.value
        except:
            print(f"Run went wrong")
            return None
    
    

In [10]:
dfs[0]['support'][0]

'Less incentive to study at university'

In [16]:
new_filtered = ["Traditional university has to be replaced by online learning",
                "There should be a presumption in favour of publication for information held by public bodies",
                "There must be a system of universal healthcare",
                "The Private Education System Should Be Abolished",
                "Provide breakfast for all school students"
                ]

In [18]:
for i in tqdm(range(len(dfs))):
    print(new_filtered[i])
    data = dfs[i]
    data['Chat_gpt'] = 0
    for j in tqdm(range(len(data['support']))):
        # print(data['Texts'][i])
        text1 = data['support'][j]
        for k in range(len(data['against'])):
            text2 = data['against'][k]
            if data['topic_support'][j] != data['topic_against'][k]:
                res = run_inconsistency_detection(text1, text2)
                data['Chat_gpt'][j] = res
                data.to_csv(f"results/OESD_perspectrum_40/with_gpt_comments/{new_filtered[i]}.csv", index=False)





  0%|          | 0/5 [00:00<?, ?it/s]

Traditional university has to be replaced by online learning


C:\Users\Nursulu_1\AppData\Local\Temp\ipykernel_419244\1974346899.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Chat_gpt'][j] = res
C:\Users\Nursulu_1\AppData\Local\Temp\ipykernel_419244\1974346899.py:12: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'Reasoning: Text 1 promotes the accessibility of academic research through social media which broadens the educational resources available to the public. Text 2 points out a potential concern regarding revenue for universities, suggesting that making too much content free can impact their ability to secure private funding. There doesn't necessarily appear to be a direct inconsistency between these two texts as one discusses the benefits of broader access through social medi

There should be a presumption in favour of publication for information held by public bodies


C:\Users\Nursulu_1\AppData\Local\Temp\ipykernel_419244\1974346899.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Chat_gpt'][j] = res
C:\Users\Nursulu_1\AppData\Local\Temp\ipykernel_419244\1974346899.py:12: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'Reasoning: Text 1 emphasizes the importance of making information accessible to all individuals, including non-citizens and minors, to ensure they are informed about matters that affect them. Text 2, however, speaks about restricting the publication of specific content, such as "torturous scenes." These texts address different aspects of information dissemination—Text 1 about broad access and Text 2 about content restriction. There is no specific inconsistency as they deal

...

C:\Users\Nursulu_1\AppData\Local\Temp\ipykernel_419244\1974346899.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Chat_gpt'][j] = res
C:\Users\Nursulu_1\AppData\Local\Temp\ipykernel_419244\1974346899.py:12: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'Reasoning: Redistributing wealth to balance income disparity, as stated in Text 1, usually involves using increased tax revenues or similar means to fund policies that aim to reduce economic inequality. Universal healthcare, which is mentioned in Text 2 as being unaffordable, is a type of policy that could potentially be funded through redistribution of wealth. If Text 1 advocates for redistribution to tackle inequality, it could logically imply support for funding policie

The Private Education System Should Be Abolished


C:\Users\Nursulu_1\AppData\Local\Temp\ipykernel_419244\1974346899.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Chat_gpt'][j] = res
C:\Users\Nursulu_1\AppData\Local\Temp\ipykernel_419244\1974346899.py:12: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'Unrelated.' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  data['Chat_gpt'][j] = res
 80%|████████  | 4/5 [21:44<05:45, 345.59s/it]

Provide breakfast for all school students


C:\Users\Nursulu_1\AppData\Local\Temp\ipykernel_419244\1974346899.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Chat_gpt'][j] = res
C:\Users\Nursulu_1\AppData\Local\Temp\ipykernel_419244\1974346899.py:12: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'Answer: No

Reasoning: Both texts emphasize ensuring well-being through adequate resources; Text 1 speaks about resources for a healthy life broadly, and Text 2 specifies one aspect of this principle by focusing on parents ensuring their children have a nutritious breakfast. Neither statement contradicts the other; rather, Text 2 provides a specific action that supports the broader principle outlined in Text 1.' has dtype incompatible with int64, please explicitly cast to 

In [38]:
data[data['text']=="Standardized tests keep schools accountable in making sure their students are prepared for the world."]['target']

381    Abolish standardized tests for University Admi...
Name: target, dtype: object

In [41]:
# Extract the target values for the texts in support
targets = data[data['text'].isin(support)]['target'].tolist()

targets

['Airbrushing of women’s bodies in the media be banned',
 'Airbrushing of women’s bodies in the media be banned',
 'Co2 Does Not Cause Global Warming',
 'Co2 Does Not Cause Global Warming',
 'Co2 Does Not Cause Global Warming',
 'Co2 Does Not Cause Global Warming',
 'Co2 Does Not Cause Global Warming',
 'Co2 Does Not Cause Global Warming',
 'Co2 Does Not Cause Global Warming',
 'Co2 Does Not Cause Global Warming',
 'Cease the exploitation of the Arctic Region']

In [40]:
target_list

['Airbrushing of women’s bodies in the media be banned',
 'Airbrushing of women’s bodies in the media be banned',
 'Co2 Does Not Cause Global Warming',
 'Co2 Does Not Cause Global Warming',
 'Co2 Does Not Cause Global Warming',
 'Co2 Does Not Cause Global Warming',
 'Co2 Does Not Cause Global Warming',
 'Co2 Does Not Cause Global Warming',
 'Co2 Does Not Cause Global Warming',
 'Co2 Does Not Cause Global Warming',
 'Cease the exploitation of the Arctic Region']

In [34]:
df = pd.read_csv("results/OESD_perspectrum_40/Co2 Does Not Cause Global Warming.csv")

In [32]:
df['topic_support'][0].loc[216]

AttributeError: 'str' object has no attribute 'loc'

### Checking degrees of confidence

In [6]:
client = OpenAI()

response = client.completions.create(
  model="gpt-3.5-turbo-instruct",
  prompt=("Your task is to. You see that Timmy likes to step on cat's tails to hear them howl. Is Timmy:"
  "1: Naughty? 2: Nice?\n\n"
  "(Santa Claus only outputs the number as his answer.)\n\nA: "),
  temperature=2,
  max_tokens=1,
  logprobs=5,
)
log_probs = response.choices[0].logprobs.top_logprobs[0]
normal_probs = {key: math.exp(value) for key, value in log_probs.items()}
print(normal_probs)

{'1': 0.9985837425405054, ' ': 0.0005641609358038662, '\n': 0.00048636762012253045, '2': 0.00019892235206740561, '\n\n': 5.4404102137826425e-05}


In [37]:
client = OpenAI()

response = client.completions.create(
  model="gpt-3.5-turbo-instruct",
  prompt=("Your task is to tell whether two phrases can be used interchangeably without losing the meaning."
          "Phrase 1: childcare_subsidies. Phrase 2: childcare_investments"
  "(You can only output 'yes' or 'no' as an answer.) A: "),
  temperature=2,
  max_tokens=1,
  logprobs=5,
)
log_probs = response.choices[0].logprobs.top_logprobs[0]
normal_probs = {key: math.exp(value) for key, value in log_probs.items()}
print(normal_probs)

{'no': 0.3915460679905344, 'No': 0.32816078255032416, '\n': 0.11634836985877663, '\n\n': 0.04509597421424368, ' no': 0.030935923423506814}
